In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import re
import os
import shutil
import glob
import zipfile
import cv2

In [39]:
path= '골밀도 데이터/BoneAge/'
data_path = '골밀도 데이터/image/'

# xlsx파일과 zip파일 경로 추출
xls_path=glob.glob(path+"/*.xlsx")
zip_path=glob.glob(path+"/*.zip")

In [40]:
#zip파일 압축 해제 후 위치 변경
for zip in zip_path:
    zipfile.ZipFile(zip).extractall(data_path)

In [41]:
#이중 폴더 내 이미지 꺼내기
folder_path=glob.glob(data_path+"BA*")
for in_path in folder_path:
    tmp=os.listdir(in_path)
    for i in tmp:
        os.rename(in_path+"/"+i,data_path+str(int(i[0:3]))+i[3:])
    
    os.rmdir(in_path)

In [42]:
#xlsx파일 읽어서 합치기
df=pd.DataFrame()
for xlsx in xls_path:
    sub_df = pd.read_excel(xlsx)
    
    id = pd.DataFrame([int(re.findall(r'\d+', str(x))[0]) for x in sub_df["No."]])
    age = sub_df["검사 시 나이"]
    gender = sub_df["성별"]-1
    
    tmp = pd.concat([id, age, gender], axis=1, ignore_index=True)
    df = pd.concat([df, tmp], axis=0, ignore_index=True)
    
df.columns=["ID","Age","Gender"]

In [43]:
df

,ID,Age,Gender
0,310,9.969863,1
1,311,9.989041,1
2,312,10.008219,1
3,313,10.049315,1
4,314,10.060274,1
...,...,...,...
1232,327,9.860274,0
1233,328,9.863014,0
1234,329,9.868493,0
1235,330,9.868493,0


In [44]:
#labeling 데이터 저장
df.to_excel(data_path+"clin_info.xlsx")